In [1]:
import requests
from tqdm import tqdm
import math
import zipfile
from aux_functions import tqa_data_extraction, tqa_data_refiner, get_tqa_model, get_tqa_sequence_lengths, get_dropouts
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, precision_score, recall_score
from keras import optimizers
from sklearn.metrics import classification_report

/home/test/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


  # Download TQA

In [ ]:
url = "https://s3.amazonaws.com/ai2-vision-textbook-dataset/dataset_releases/tqa/tqa_train_val_test.zip"

r = requests.get(url, stream=True)
total_size = int(r.headers.get('content-length', 0))
block_size = 1024
wrote = 0 
with open("tqa_train_val_test.zip", 'wb') as f:
  for data in tqdm(r.iter_content(block_size), total=math.ceil(total_size//block_size) , unit='KB', desc = "tqa_train_val_test.zip", leave = True):
    wrote = wrote  + len(data)
    f.write(data)
if total_size != 0 and wrote != total_size:
  print("ERROR, something went wrong")
f.close
zip_ref = zipfile.ZipFile("tqa_train_val_test.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

In [2]:
!ls "./tqa_train_val_test"

CVPR17_TQA.pdf	README.md  test  train	val


# Corpus loading

In [3]:
question_type = "diagramQuestions"
#question_type = "nonDiagramQuestions"

weights = "cross"
#weights = "cross-vecsi"

In [4]:
train_folder = "./tqa_train_val_test/train/"
train_json = "./tqa_train_val_test/train/tqa_v1_train.json"
test_folder = "./tqa_train_val_test/test/"
test_json = "./tqa_train_val_test/test/tqa_v2_test.json"
val_folder = "./tqa_train_val_test/val/"
val_json = "./tqa_train_val_test/val/tqa_v1_val.json"

dataset_jsons = [train_json,test_json,val_json]
dataset_folders = [train_folder,test_folder,val_folder]

dim = 100
epochs = 10
batchSize = 128
dout, rdout = get_dropouts(question_type)
context_length, question_length = get_tqa_sequence_lengths(question_type)

## Data loading

In [5]:
data, figures, correct_answers = tqa_data_extraction(dataset_folders, dataset_jsons, question_type)

#Data_refining
texts = []

for i in range(len(data[0])):
    for j in range(len(data)):
        texts.append(data[j][i])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Get the vocabulary index
word_index = tokenizer.word_index
print("Found %s unique tokens." % len(word_index))

x, y = tqa_data_refiner(data, figures, correct_answers,tokenizer, weights, context_length, question_length)

1076 lessons processed

Found 9268 unique tokens.
Shape of (0) data tensor:(12567, 418)
12567/12567 [==============================] - 43s 3ms/step
Shape of (1) data tensor:(12567, 1, 1, 512)
Shape of (2) data tensor:(12567, 46)
12567/12567 [==============================] - 34s 3ms/step
Shape of (3) data tensor:(12567, 1, 1, 512)
Shape of (4) data tensor:(12567, 46)
Shape of (5) data tensor:(12567, 46)
Shape of (6) data tensor:(12567, 46)
Shape of (7) data tensor:(12567, 46)
Shape of labels tensor: (12567, 4)




In [6]:
kfold = KFold(n_splits=10, shuffle=True)
precisions = []
recalls = []
f1s = []
n_words = len(word_index)

for train, test in kfold.split(x[0], y):
    
    x_train = []
    x_test = []
    for elem in x:
        x_train.append(elem[train])
        x_test.append(elem[test])
    y_train = y[train]
    y_test = y[test]
    
    model = get_tqa_model(n_words, dim, dout, rdout, context_length, question_length)
    
    adam = optimizers.Adam(lr=1e-2, decay=0.0)
    model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=['categorical_accuracy'])
    model.fit(x_train, y_train, batch_size=batchSize, epochs=epochs, validation_data=(x_test, y_test), verbose=1)
    
    pred = model.predict(x_test, batch_size=batchSize)
    max_value = np.argmax(pred,axis=1)
    predNew = np.zeros(np.shape(pred))
    for i in range(len(predNew)):
        predNew[i,max_value[i]]=1
    print(classification_report(y_test, predNew, digits=4, target_names=["a","b","c","d"]))
    precisions.append(precision_score(y_test, predNew, average="weighted"))
    recalls.append(recall_score(y_test, predNew, average="weighted"))
    f1s.append(f1_score(y_test, predNew, average="weighted"))
    
print("Precision: %.2f (+/- %.2f)" % (np.mean(precisions), np.std(precisions)))
print("Recall: %.2f (+/- %.2f)" % (np.mean(recalls), np.std(recalls)))
print("F1 Score: %.2f (+/- %.2f)" % (np.mean(f1s), np.std(f1s)))
    

Train on 11310 samples, validate on 1257 samples
Epoch 1/10
11310/11310 [==============================] - 51s 5ms/step - loss: 1.4771 - categorical_accuracy: 0.2437 - val_loss: 1.3973 - val_categorical_accuracy: 0.2617
Epoch 2/10
 1664/11310 [===>..........................] - ETA: 37s - loss: 1.3759 - categorical_accuracy: 0.2885

KeyboardInterrupt: 